<a href="https://colab.research.google.com/github/StevenMElliott/DS-Unit-2-Applied-Modeling/blob/master/CAT_Tube_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
SOURCE = '../data/caterpillar/caterpillar-tube-pricing/competition_data/'

In [2]:
!wget https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/caterpillar/caterpillar-tube-pricing.zip

--2019-07-24 21:40:37--  https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/caterpillar/caterpillar-tube-pricing.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 823789 (804K) [application/zip]
Saving to: ‘caterpillar-tube-pricing.zip.1’

caterpillar-tube-pr 100%[===================>] 804.48K  --.-KB/s    in 0.04s   

2019-07-24 21:40:37 (18.0 MB/s) - ‘caterpillar-tube-pricing.zip.1’ saved [823789/823789]



In [3]:
!unzip caterpillar-tube-pricing.zip

Archive:  caterpillar-tube-pricing.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [4]:
!unzip data.zip

Archive:  data.zip
replace competition_data/bill_of_materials.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [0]:
SOURCE = 'competition_data/'

In [0]:
import pandas as pd
test = pd.read_csv(SOURCE + 'test_set.csv')
trainval = pd.read_csv(SOURCE + 'train_set.csv')

In [7]:
trainval.head()

,tube_assembly_id,supplier,quote_date,annual_usage,min_order_quantity,bracket_pricing,quantity,cost
0,TA-00002,S-0066,2013-07-07,0,0,Yes,1,21.905933
1,TA-00002,S-0066,2013-07-07,0,0,Yes,2,12.341214
2,TA-00002,S-0066,2013-07-07,0,0,Yes,5,6.601826
3,TA-00002,S-0066,2013-07-07,0,0,Yes,10,4.687770
4,TA-00002,S-0066,2013-07-07,0,0,Yes,25,3.541561


In [8]:
df = pd.read_csv(SOURCE + 'specs.csv')
df.head()

,tube_assembly_id,spec1,spec2,spec3,spec4,spec5,spec6,spec7,spec8,spec9,spec10
0,TA-00001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TA-00002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TA-00003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TA-00004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TA-00005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df.describe()

,tube_assembly_id,spec1,spec2,spec3,spec4,spec5,spec6,spec7,spec8,spec9,spec10
count,21198,7129,6844,5840,4154,2921,2071,535,106,20,1
unique,21198,45,51,49,38,37,24,13,4,3,1
top,TA-12111,SP-0007,SP-0012,SP-0080,SP-0026,SP-0080,SP-0082,SP-0082,SP-0082,SP-0088,SP-0080
freq,1,3002,1736,1798,1492,1780,1167,322,73,18,1


In [20]:
# Group the tube assemblies by assembly ID to prevent mixing them.
trainval_tube_assemblies = trainval['tube_assembly_id'].unique()
test_tube_assemblies = test['tube_assembly_id'].unique()

# Split trainval
from sklearn.model_selection import train_test_split
train_tube_assemblies, val_tube_assemblies = train_test_split(
    trainval_tube_assemblies, random_state=42
)

#
train = trainval[trainval['tube_assembly_id'].isin(train_tube_assemblies)]
val = trainval[trainval['tube_assembly_id'].isin(val_tube_assemblies)]
assert(len(train) + len(val) == len(trainval))
train.shape, val.shape, trainval.shape

((22628, 8), (7585, 8), (30213, 8))

In [0]:
assert(len(train) + len(val) == len(trainval))

In [0]:
def wrangle(X):
  
  #Date Features
  X['quote_date'] = pd.to_datetime(X['quote_date'], infer_datetime_format=True)
  X['quote_date_year'] = X['quote_date'].dt.year
  X['quote_date_year'] = X['quote_date'].dt.month
  #Now that we have the year and date, we don't need the whole quote date
  X = X.drop(columns='quote_date')
  
  #Merge tube data
  tube = pd.read_csv(SOURCE + 'tube.csv')
  X = X.merge(tube, how='left')
  
  #Merge bill of materials data
  mat = pd.read_csv(SOURCE + 'bill_of_materials.csv')
  X = X.merge(mat, how='left')
  
  #The goal is to predict UNK tube assemblies. This needs to be at the end
  #beacause I want to merge stuff first.
  X = X.drop(columns='tube_assembly_id')
  return X

In [0]:
# test_wrangled = wrangle(test)
# trainval_wrangled = wrangle(trainval)

In [0]:
trainval_wrangled.head(30)